<a href="https://colab.research.google.com/github/daa103/DAA_ESAA/blob/main/%EB%AA%A8%EB%8D%B8%ED%9B%88%EB%A0%A8_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___


- 확률적 경사하강법: 훈련세트 중 한 샘플을 무작위로 선택하여 샘플에 대한 그래디언트 계산. 하나의 샘플만 있으면 되므로 큰 훈련세트도 훈련 가능.
- 미니 배치 경사 하강법: 미니배치라고 부르는 작은 샘플 세트에 대해 그래디언트를 계산
- 그리드 탐색과정에서 반복횟수를 제한하면 배치 경사하강법도 가능하다.

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

- 학습률이 높아 검증오차 발산
- 모델의 과대적합
> - 더 많은 훈련데이터를 추가
> - 학습률 낮추기
> - 훈련 멈추기

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

- alpha는 모델을 얼마나 규제할지 조절하는데, alpha가 아주 크면 모든 가중치가 0에 가까워진다고 보면된다.
- 과대적합(overfitting)은 모델이 훈련 데이터에 너무 맞춰져서 훈련 오차는 낮지만 검증 오차가 높은 경우 발생
- 과소적합은모델이 훈련 데이터에도 제대로 적합하지 않고, 새로운 데이터에 대한 일반화 능력이 부족하다는 의미
- 훈련오차와 검증오차가 비슷하고 둘다 높으면 과소적합. 이는 편향이 높다는 것을 의미
- 높은 편향은 모델이 너무 단순하거나 제한적이어서 데이터의 다양성을 포착하지 못하는 것을 나타냅니다.
- 편향이 높으면 과소적합, 분산이 높으면 과대적합이므로 높은 편향이 문제이다.
- 규제 하이퍼파라미터 alpha를 증가시킬수록 직선에 가까워지고 모델의 분산은 줄고 편향이 커지므로 줄여야 한다.

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

> - 과대적합을 감소시키기 위해 모델을 규제하는데, 규제가 없는 모델인 선형회귀의 단점을 보완하기 위해 규제가 있는 릿지회귀를 사용한다. 모델훈련이 끝나면 성능은 규제가 없는 성능지표로 평가한다.
> - 라쏘회귀는 덜 중요한 특성의 가중치를 제거하려는 점에서 릿지회귀와 다르다.
> - 엘라스틱넷은 라쏘와 릿지의 절충안으로, 특성수가 훈련샘플수보다 많거나 특성들이 강한 연관성을 보일 때 라쏘는 문제가 발생하여 엘라스틱넷을 사용한다.

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [1]:
#챗gpt코드
import numpy as np

class SoftmaxRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000, early_stopping=True, tol=1e-4):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.early_stopping = early_stopping
        self.tol = tol

    def fit(self, X, y):
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y))

        # One-hot encoding of target labels
        y_one_hot = self.one_hot_encode(y, n_classes)

        # Initialize weights and biases
        self.theta = np.zeros((n_features, n_classes))
        self.bias = np.zeros(n_classes)

        prev_loss = float('inf')

        for i in range(self.n_iterations):
            logits = np.dot(X, self.theta) + self.bias
            probabilities = self.softmax(logits)

            # Calculate gradient
            gradient = np.dot(X.T, (probabilities - y_one_hot)) / n_samples
            bias_gradient = np.mean(probabilities - y_one_hot, axis=0)

            # Update weights and biases
            self.theta -= self.learning_rate * gradient
            self.bias -= self.learning_rate * bias_gradient

            # Compute loss
            loss = self.cross_entropy_loss(probabilities, y_one_hot)

            # Early stopping
            if self.early_stopping and abs(prev_loss - loss) < self.tol:
                print(f"Converged at iteration {i}.")
                break

            prev_loss = loss

    def predict(self, X):
        logits = np.dot(X, self.theta) + self.bias
        probabilities = self.softmax(logits)
        return np.argmax(probabilities, axis=1)

    def one_hot_encode(self, y, n_classes):
        one_hot = np.zeros((len(y), n_classes))
        for i, label in enumerate(y):
            one_hot[i, label] = 1
        return one_hot

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def cross_entropy_loss(self, y_pred, y_true):
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        return -np.sum(y_true * np.log(y_pred)) / len(y_true)

# Example usage
if __name__ == "__main__":
    from sklearn.datasets import load_iris
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import accuracy_score

    # Load the Iris dataset
    iris = load_iris()
    X, y = iris.data, iris.target

    # Standardize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create and fit the Softmax Regression model
    model = SoftmaxRegression(learning_rate=0.1, n_iterations=1000, early_stopping=True, tol=1e-4)
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")


Converged at iteration 737.
Accuracy: 1.00
